# CIFAR-10 Convolutional Neural Networks Assignment

## Learning Objectives
- Understand convolutional layers and their inductive bias
- Analyze architectural design choices (kernel size, depth, stride, padding)
- Compare CNNs with fully connected baseline models
- Deploy the model to AWS SageMaker

## Dataset: CIFAR-10
CIFAR-10 is a 32x32 RGB image dataset with 10 classes (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck). It contains 50,000 training images and 10,000 test images.

In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Load Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print("Training data shape:", x_train.shape)
print("Testing data shape:", x_test.shape)

# Visualize Sample Images
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_train[i])
    plt.title(f'Label: {y_train[i][0]}')
    plt.axis('off')
plt.show()

In [ ]:
# Prepare Data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten Data for Baseline
x_train_flat = x_train.reshape(-1, 32 * 32 * 3)
x_test_flat = x_test.reshape(-1, 32 * 32 * 3)

# Baseline Model (Dense only)
baseline_model = Sequential([
    Dense(512, activation='relu', input_shape=(32 * 32 * 3,)),
    Dense(10, activation='softmax')
])

baseline_model.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

baseline_model.fit(x_train_flat, y_train,
                   epochs=10,
                   validation_data=(x_test_flat, y_test))

In [ ]:
# CNN Model
cnn_model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model.fit(x_train, y_train,
              epochs=10,
              validation_data=(x_test, y_test))

In [ ]:
# Controlled Experiments on Kernel Size
kernel_sizes = [(3, 3), (5, 5), (7, 7)]
for size in kernel_sizes:
    print(f"\nTraining with kernel size {size}")
    model = Sequential([
        Conv2D(32, kernel_size=size, activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              epochs=5,
              validation_data=(x_test, y_test))

In [ ]:
# Placeholder for SageMaker Deployment
import sagemaker
from sagemaker.tensorflow import TensorFlow

# Example skeleton (requires AWS setup):
# estimator = TensorFlow(entry_point='train.py',
#                        role='SageMakerRole',
#                        instance_type='ml.m5.large',
#                        framework_version='2.3',
#                        py_version='py37')
# estimator.fit({'training': 's3://bucket/cifar10/train'})
# predictor = estimator.deploy(initial_instance_count=1,
#                              instance_type='ml.m5.large')